# Word Count of a Song 
Listing all of the unique words in a song and displays the count of each unique word

## What it does: 
* Get the lyrics of a song given the song title and artist name. If not available, paste the whole lyrics in the input
* List the words of the requested song and how many times they appeared throughout the song.
 
*You will need an internet connection to search for the lyrics of the song requested*


## Defining a function that sends a request to Genius API

We get the lyrics of the song from Genius.com so we need first to define a function that makes a request to Genius. This function takes the song,artist name, and access token and returns the response.


* To get access to the API, register/login to Genius.com.<br>*For the purpose of this notebook, I have already created a test account and generated an access token.<br> It would be nice to put in place better procedure authentication in the future.*


* Notes in getting the access token:
  * Genius API : https://genius.com/developers
  * When creating an API Client, it will ask for an application URL. <br>For the purpose of running scripts in this notebook, I used http://example.com as the url.
  


In [1]:
import requests #Requests module is used here to make requests to an API.

def request_song_info(song_title, artist_name,access_token):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + access_token} 
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

## Access Token

Below is the generated access token which we wil pass to the function

In [2]:
access_token = '_g6VbArTrxw0IVQcrs4KwOlyx4SFjxzJ1ujigZ_uHakQp1VKJS1b3FrqtizUIm3P' #access token from test account

## Song Title and Artist Name

This asks for input on the song title and artist name. In order for it to be very accurate, please input both correctly. 


In [3]:

song_title_search = input("Enter song title:")
artist_name_search = input("Enter song artist:")


Enter song title:Love
Enter song artist:Kendrick Lamar


## Getting the Song Lyrics URL from the Response

This calls the previously defined function to send a request to the API. <br>
The API sends a response and we look through the response for a match in the artist name and the matching url for that artist.<br>

In [4]:
import json

#call function to get response
response = request_song_info(song_title_search, artist_name_search,access_token) 

#converts response to a dictionary
response_dict = response.json() 

#variable for storing song url found in the response
song_info = None 

if response_dict['response']['hits'] ==' ':
    lyrics_input=input("Paste lyrics: ")
    
#look for primary artist and get song info
for hit in response_dict['response']['hits']:
    if artist_name_search.lower() in hit['result']['primary_artist']['name'].lower():
        song_info = hit
        break

#get song url
if song_info:
    song_url = song_info['result']['url']
    full_title = song_info['result']['full_title']


## Scrape the lyrics from the url

Next step is to scrape the lyrics of the song using BeautifulSoup.

In [5]:
# Scrape lyrics
from bs4 import BeautifulSoup

#function for scaraping lyrics
def scrape_lyrics(url):
    #send request to get lyrics page of the url input
    page = requests.get(url) 
    #Beautiful Soup
    html = BeautifulSoup(page.text, 'html.parser')
    #
    lyrics = html.find('div', class_='lyrics').get_text()

    return lyrics


## Song Lyrics

Print the lyrics of the song.

In [6]:
lyrics = scrape_lyrics(song_url)

print("[{}] \n {}".format(full_title,lyrics))

[LOVE. by Kendrick Lamar (Ft. Zacari)] 
 

[Intro: Kendrick Lamar]
Damn, love or lust
Damn, all of us

[Chorus: Zacari & Kendrick Lamar]
Give me a run for my money
There is nobody, no one to outrun me
(Another world premiere!)
So give me a run for my money
Sippin' bubbly, feelin’ lovely, livin' lovely
Just love me
I wanna be with you, ayy, I wanna be with
Just love me, just love me, just love
I wanna be with you, ayy, I wanna be with
Love me
I wanna be with you
Love me, just love me

[Refrain: Kendrick Lamar & Zacari]
If I didn't ride blade on curb, would you still love me?
If I minimized my net-worth, would you still love me?
Keep it a hundred, I’d rather you trust me than to love me
Keep it a whole one hund': don't got you, I got nothin'

[Verse 1: Kendrick Lamar]
Ayy, I got somethin'
Hol' up, we gon' function, ayy, no assumptions, ayy
Feelin' like Tyson with it
Knock it out twice, I’m with it
Only for the night, I’m kiddin'
Only for life, yeah, only for life, yeah
Only for life, let

## Remove Comments and get only alphanumeric words
We use regex to remove words inside brackets which are comments in the song

In [73]:
import re

lyrics_no_comments = re.sub("[\(\[].*?[\)\]]|(?!\b'\b)|[,!?.]", "", lyrics)
print(lyrics_no_comments)




Damn love or lust
Damn all of us


Give me a run for my money
There is nobody no one to outrun me

So give me a run for my money
Sippin' bubbly feelin’ lovely livin' lovely
Just love me
I wanna be with you ayy I wanna be with
Just love me just love me just love
I wanna be with you ayy I wanna be with
Love me
I wanna be with you
Love me just love me


If I didn't ride blade on curb would you still love me
If I minimized my net-worth would you still love me
Keep it a hundred I’d rather you trust me than to love me
Keep it a whole one hund': don't got you I got nothin'


Ayy I got somethin'
Hol' up we gon' function ayy no assumptions ayy
Feelin' like Tyson with it
Knock it out twice I’m with it
Only for the night I’m kiddin'
Only for life yeah only for life yeah
Only for life let’s get it
Hit that shoulder lean
I know what comin' over me
Backstroke oversea
I know what you need
Already on ten all money come in
All feeling go out this feeling don't drought
This party won't end


If I did

## Defining a function for counting occurences of each unique word

The function splits the lyrics into a list of words and creates a dictionary of count of occurences by iterating through the list 

In [74]:

def word_count(str):
    
    #split strings into a list, make words to have the same case
    words = str.lower().split()

    #create a dictionary of word counts
    counts = dict()
    
    #iterates through the list to count each occurence
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    
    #returns the dictionary output
    return counts


## Finally count the words :)
Call the function to get a dictionary of the count per word and then print it.

In [75]:
#call function to get the word count
counts = word_count(lyrics_no_comments)

#printing the dictionar
for key, value in sorted(counts.items(), key=lambda item: item[1],reverse=True):
    print("%s: %s" % (key, value))

me: 43
i: 36
love: 31
you: 26
with: 17
be: 16
just: 15
wanna: 15
a: 13
the: 13
it: 12
for: 11
my: 9
to: 9
ayy: 9
on: 9
got: 8
way: 8
money: 7
one: 7
give: 6
run: 6
lovely: 6
if: 6
would: 6
still: 6
keep: 6
no: 5
don't: 5
i'm: 5
your: 5
is: 4
only: 4
know: 4
all: 3
there: 3
nobody: 3
outrun: 3
so: 3
sippin': 3
bubbly: 3
livin': 3
ride: 3
blade: 3
curb: 3
minimized: 3
net-worth: 3
hundred: 3
rather: 3
trust: 3
than: 3
whole: 3
hund':: 3
nothin': 3
we: 3
feelin': 3
life: 3
that: 3
what: 3
damn: 2
didn't: 2
up: 2
like: 2
out: 2
i’m: 2
yeah: 2
in: 2
feeling: 2
this: 2
i'd: 2
want: 2
by: 2
told: 2
from: 2
yo': 2
or: 1
lust: 1
of: 1
us: 1
feelin’: 1
i’d: 1
somethin': 1
hol': 1
gon': 1
function: 1
assumptions: 1
tyson: 1
knock: 1
twice: 1
night: 1
kiddin': 1
let’s: 1
get: 1
hit: 1
shoulder: 1
lean: 1
comin': 1
over: 1
backstroke: 1
oversea: 1
need: 1
already: 1
ten: 1
come: 1
go: 1
drought: 1
party: 1
won't: 1
end: 1
didn’t: 1
ain't: 1
time: 1
waste: 1
poppin': 1
gum: 1
am: 1
wan': 1
pressure: